In [9]:
from __future__ import print_function
import matplotlib.pyplot as plt  
import matplotlib.patches as patches
%matplotlib inline
from IPython import display as dp
import numpy as np
from skimage import io
import os
import time
import cv2
from sort import *
from utils.util import load_from_txt

ImportError: cannot import name 'Sort' from 'sort' (unknown location)

In [2]:
fileDetections = "/ghome/group03/dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt"

colours = np.random.rand(32,3) #used only for display
detections = load_from_txt(fileDetections,threshold=0.5) #load detections


display = True
total_time = 0.0
total_frames = 0
out = []
if display:
    plt.ion() # for iterative display
    fig, ax = plt.subplots(1, 2,figsize=(20,20))

mot_tracker = Sort() #create instance of the SORT tracker

for frame in range(int(detections[:,0].max())): # all frames in the sequence
    frame += 1 #detection and frame numbers begin at 1
    dets = detections[detections[:,0]==frame,2:7]   
    
    if (display):
        fn = '../../../data/2DMOT2015/%s/%s/img1/%06d.jpg'%(phase,seq,frame) # read the frame
        im =io.imread(fn)
        ax[0].imshow(im)
        ax[0].axis('off')
        ax[0].set_title('Original Faster R-CNN detections')
        for j in range(np.shape(dets)[0]):
            color = colours[j]
            coords = (dets[j,0],dets[j,1]), dets[j,2], dets[j,3]
            ax[0].add_patch(plt.Rectangle(*coords,fill=False,edgecolor=color,lw=3))
            
    dets[:,2:4] += dets[:,0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2] for the tracker input
    total_frames += 1

    if(display):
        ax[1].imshow(im)
        ax[1].axis('off')
        ax[1].set_title('Tracked Targets')

    start_time = time.time()
    trackers = mot_tracker.update(dets)
    cycle_time = time.time() - start_time
    total_time += cycle_time
    
    out.append(trackers)
    for d in trackers:
        if(display):
            d = d.astype(np.uint32)
            ax[1].add_patch(patches.Rectangle((d[0],d[1]),d[2]-d[0],d[3]-d[1],fill=False,lw=3,ec=colours[d[4]%32,:]))
            ax[1].set_adjustable('box-forced')

    if(display):
        dp.clear_output(wait=True)
        dp.display(plt.gcf())
        time.sleep(0.000001)
        ax[0].cla()
        ax[1].cla()

print("Total Tracking took: %.3f for %d frames or %.1f FPS"%(total_time,total_frames,total_frames/total_time))


In [4]:
video_capture = cv2.VideoCapture('C:/Users/AnaHarris/Documents/MASTER/M6/project/dataset/AICity_data/train/S03/c010/vdo.avi')

In [6]:
def track_memory(tracked_objects):
    delete = []
    for idx in tracked_objects:
        if tracked_objects[idx]['memory'] > 0:
            tracked_objects[idx]['memory'] -= 1
        else:
            delete.append(idx)

    for idx in delete:
        del tracked_objects[idx]

In [7]:
track_id = 0
tracked_objects = {}
memory = 10
for frame_id in mask_rcnn_boxes:
    # REMOVE OVERLAPPING BOUNDING BOXES
    boxes = mask_rcnn_boxes[frame_id]
    frame_boxes = discard_overlaps(boxes)
    # FIRST FRAME, WE INITIALIZE THE OBJECTS ID
    if frame_id == 0:
        for j in range(len(frame_boxes)):
            # We add the tracking object ID at the end of the list  [[frame,x1, y1, x2, y2, conf, track_id]]
            frame_boxes[j].append(track_id)
            tracked_objects[f'{track_id}'] = {'bbox':[frame_boxes[j][1],frame_boxes[j][2],frame_boxes[j][3],frame_boxes[j][4]],'frame':frame_id,'memory':memory}
            track_id += 1
            
    else:
        # FRAME N+1 WE COMPARE TO OBJECTS IN FRAME N
        for i in range(len(frame_boxes)):
            best_iou = 0
            track_id_best = 0
            boxA = [frame_boxes[i][1],frame_boxes[i][2],frame_boxes[i][3],frame_boxes[i][4]]

            for data in tracked_objects.items():
                id,boxB = data
                iou_score,_ = iou(boxA,boxB['bbox'])

                if iou_score > best_iou:
                    best_iou = iou_score
                    track_id_best = id

            if track_id_best == 0 and best_iou == 0:
                frame_boxes[i].append(track_id)
                tracked_objects[f'{track_id}']  = {'bbox':boxA,'frame':frame_id,'memory':memory}
                track_id += 1
                
            else:
                if tracked_objects[f'{track_id_best}']['frame'] == frame_id:
                    frame_boxes[i].append(track_id)
                    tracked_objects[f'{track_id}']  = {'bbox':boxA,'frame':frame_id,'memory':memory}
                    track_id += 1
                else:
                    frame_boxes[i].append(track_id_best)
                    tracked_objects[f'{track_id_best}']['bbox']= boxA
                    tracked_objects[f'{track_id_best}']['frame'] = frame_id
                    
                    
    track_memory(tracked_objects)
            

In [8]:
for i in range(0,100):#len(mask_rcnn_boxes)):
    frame_visualization(video_capture,mask_rcnn_boxes,i,'C:/Users/AnaHarris/Documents/GitHub/mcv-m6-2023-team6/week3/Results/task2_1','mask_rcnn')

In [15]:
mask_rcnn_boxes[2140]

[[2140, 933.41, 80.331, 1013.9639999999999, 141.946, 0.622, '422'],
 [2140, 896.145, 94.306, 939.048, 141.194, 0.794, '424'],
 [2140, 1175.69, 101.09, 1227.8120000000001, 156.80700000000002, 0.945, '1'],
 [2140, 916.409, 120.301, 1011.846, 200.375, 0.988, '396'],
 [2140, 1286.74, 360.295, 1514.592, 537.0350000000001, 0.998, '3']]

In [14]:
tracked_objects

{'1': {'bbox': [1175.69, 101.09, 1227.8120000000001, 156.80700000000002],
  'frame': 2140,
  'memory': 4},
 '3': {'bbox': [1286.74, 360.295, 1514.592, 537.0350000000001],
  'frame': 2140,
  'memory': 4},
 '396': {'bbox': [916.409, 120.301, 1011.846, 200.375],
  'frame': 2140,
  'memory': 4},
 '421': {'bbox': [927.469, 87.802, 1011.8610000000001, 145.203],
  'frame': 2139,
  'memory': 3},
 '422': {'bbox': [933.41, 80.331, 1013.9639999999999, 141.946],
  'frame': 2140,
  'memory': 4},
 '423': {'bbox': [915.108, 91.516, 944.175, 138.78900000000002],
  'frame': 2136,
  'memory': 0},
 '424': {'bbox': [896.145, 94.306, 939.048, 141.194],
  'frame': 2140,
  'memory': 4}}